# Webscraping

## Intro

Dit notebook laat manieren zien hoe webpaginas te manipuleren zijn en hoe de data ervan te _scrapen_ is. 

> "_Scrapen is een computertechniek waarbij software wordt gebruikt om informatie van webpagina's te extraheren en al dan niet te analyseren._"  <sub>[wiki]<sub>

In feite maak je contact via Python om informatie op te halen van een externe informatiebron zoals een website of api om daar gegevens vandaan te halen. Deze gegevens worden vervolgens in de eigen omgeving opgeslagen. 
    
### Libraries
Python heeft een aantal [internet] tools die daarbij helpen.  
De meest gebruike zijn [urllib.request] en [html.parser].  
Deze libraries hebben een _low-level_ design waardoor het te customizen is.  
Hierdoor moet er een hoop code schrijven om de functies zo te maken dat het doel wordt bereikt.  
Gelukkig zijn er third-party libraries die een _high-level_ interface bieden.  
Python heeft ook de [json] module om objecten te encoden naar json en inkomende json te decoden.  

Er zijn veel Third-party libraries de gebruikt kunnen worden om data van het web te halen.  
Hieronder een kleine selectie waarvan we een paar in dit notebook gaat gebruiken.  

[requests] is de _high-level_ interface om [HTTP request methods] te versturen naar API's of websites.

[beautifulsoup4] wordt gebruikt om HTML of XML te parsen.  

[selenium], de meest bekende web-automation tool welk ook veel gebruikt wordt om websites te testen.  

[scrapy] een web-crawler framework voor Python met een hoop features.

[httpx] de _next-generation_ HTTP client voor Python.  
Als je bekend ben met [requests] dan is de overstap naar [httpx] klein.  
Ondersteunt HTTP2 en de Python [async] functionaliteit waar [requests] dit niet ondersteunt.


    
[internet]: https://docs.python.org/3/library/internet.html "Internet Protocols and Support"
[urllib.request]: https://docs.python.org/3/library/urllib.request.html "Extensible library for opening URLs"
[json]: https://docs.python.org/3/library/json.html "JSON encoder and decoder"
[html.parser]: https://docs.python.org/3/library/html.parser.html "Simple HTML and XHTML parser"

[httpx]: https://www.python-httpx.org/ "A next-generation HTTP client for Python"
[scrapy]: https://scrapy.org/ "Web Crawling Framework"
[selenium]: https://www.selenium.dev/documentation/en/ "selenium"
[beautifulsoup4]: https://beautiful-soup-4.readthedocs.io/en/latest/ "Beautiful Soup"
[requests]: https://docs.python-requests.org/en/master/index.html "Requests: HTTP for Humans"

[wiki]: https://nl.wikipedia.org/wiki/Scrapen
[HTTP request methods]: https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods "HTTP request methods"
[async]: https://docs.python.org/3/library/asyncio.html

## Developer tools

Naast Python gaan we ook werken met de [Developer tools] van de web browser.  
[Chrome] en [Firefox] hebben dat beiden in hun browser.  
De knop `F12` of de sneltoetsen `Shift` + `Control` + `I` opent de developer tools van de browser.  
    


[Developer tools]: https://en.wikipedia.org/wiki/Web_development_tools
[Chrome]: https://developer.chrome.com/docs/devtools/open/ "Chrome Developer tools"
[Firefox]: https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools "Firefox Developer tools"

In [1]:
# install the packages
!python -m pip install --upgrade pip requests beautifulsoup4

## Scrapen van een API

Het makelijkste om data op te vragen is doormiddel van een API. Veel websites hebben geen openbare API.

Toch kan het zo zijn dat de website een API gebruikt om data van de client naar de server te sturen.
Zo kan de website dynamisch _(zonder te refreshen)_ de data op de pagina vernieuwen.   
Hier wordt [XMLHTTPRequests (XHR)] voor gebruikt.

In de Developer Tools onder de tab Network zie je het dataverkeer en de daarbij horende HTTP request methods.  
Om goed te zien wat een website krijgt en verstuurd moet de optie Preserve log/ Persist Logs en de Disable Cache aan staan.  
Met deze opties aan behoudt de network tab alle logs en vraagt de website alle data op die anders werd onthouden in de Cache.  

---

https://www.demoblaze.com/ een demo website van [Blazemeter] en gebruikt XHR.  
In de notebook cells hieronder open we de demo website en bekijken we de Network tab voor XHR berichtenverkeer.  
En gaan we [requests] gebruiken om data van de webpagina op te vragen.  

[XMLHTTPRequests (XHR)]: https://nl.wikipedia.org/wiki/XMLHTTP
[Blazemeter]: https://www.blazemeter.com/
[requests]: https://docs.python-requests.org/en/master/index.html "Requests: HTTP for Humans"

## Oefening met developer tools
Open een browser en de developer tools.  
Zet de opties Preserve logs, Disable cache aan en filter op XHR  

> ![Developer tools](./img/devtools_XHR_filter.png "Preserve logs, Disable cache, filter XHR")

Open de demo website, we zien dat er wat XHR type requests gedaan wordt.

De url https://www.demoblaze.com/config.json wordt opgevraagt met de GET method.  

> ![Developer tools XHR get](./img/devtools_XHR_get_request.png "GET request, XHR type response")

Observeer de gegevens die te zien zijn in de response data. 

Als we er op klikken zien we de data die verstuurd en terug gekregen is.  
Onder de tab Response krijgen we de _Raw response data_ te zien.  
Dit kan van alles zijn, HTML, javascript en in dit geval JSON.  

> ![Developer tools XHR response](./img/devtools_XHR_response.png "raw response")

Hieronder wordt Python requests gebruikt om dezelfde data op te vragen.


De syntax is als volgt: 
```
responseObject = requests.get(url)
```
Nadat deze request url is uitgevoerd kun je bijvoorbeeld de pagina content ophalen via: 
```
responseObject.content
```

In [2]:
import requests
from urllib.parse import urljoin
base_url = 'https://www.demoblaze.com/'

In [3]:
path = 'config.json'
url = urljoin(base_url, path)
res = requests.get(url)

# response content is de data terug gekregen van de GET request (Raw response data)
res.content

b'{\n    "API_URL": "https://api.demoblaze.com",\n    "HLS_URL": "https://hls.demoblaze.com"\n}'

In [4]:
# HLS is een streaming protocol
# de API_URL hebben is nodig

# er is JSON data ontvangen
# de data kan geparsed worden door Python json module of door de Response.json() functie naar een dict.
config_dict = res.json()
config_dict

{'API_URL': 'https://api.demoblaze.com',
 'HLS_URL': 'https://hls.demoblaze.com'}

In [5]:
api_url = config_dict.get('API_URL')
api_url

'https://api.demoblaze.com'

Op de demo website is er een lijst te zien met product catagories.  
Wordt er op de Laptop catagorie geklikt dan wordt de Laptop catagorie opgrvraag van de server.  
Dit gebeurt met een POST request naar de API url met het _bycat_ path.  
Een POST request bevat meestal een payload of speciale headers, in dit geval is de payload een JSON string.  

> ![Catagory request met payload](./img/devtools_post_request.png "POST request met JSON payload")

In de tab Response zien we de data die verkregen is van de server.  
Dit is wederom JSON data. 

Hieronder wordt Python requests gebruikt om dezelfde data op te vragen.

In [6]:
path = 'bycat'
url = urljoin(api_url, path)

payload = {'cat': 'notebook'}

res = requests.post(url, json=payload)
notebook_dict =  res.json()
# dict structure: {'Items': [{'cat': 'notebook', id: <int>, 'img': <str>, 'price': <float>, 'title', <str>}, ... ]}

Nu de data is verkregen kan er overheen worden geitereerd.  
De data kunnen we dan opslaan in een database, csv of Excel bestand.  

In [7]:
# loop over de items in de dict
for item in notebook_dict['Items']:
    if not item.get('cat') == 'notebook':  # skip de items die geen notebook zijn.
        continue
    title = item.get('title').strip()  # strip de whitespaces en newlines van de string.
    price = item.get('price')
    print(f'{title:20} €{price}')  # maak 'title' minstens 20 characters breed

Sony vaio i5         €790.0
Sony vaio i7         €790.0
MacBook air          €700.0
Dell i7 8gb          €700.0
2017 Dell 15.6 Inch  €700.0
MacBook Pro          €1100.0


---

## Scrapen door de HTML te parsen

HTML (HyperTextMarkupLanguage) is de opmaak van een webpagina.  
De HTML kan dynamisch worden opgemaakt met (bijvoorbeeld) Javascipt code.  
Maar HTML paginas kunnen ook statisch zijn.  
Een statische pagina moet ververst worden om nieuwe data te zien.

De server die de paginas naar de client stuurt moet wel weten wat het moet doen browser (client) een pagina opvraagt.  
Als dit niet via JSON of speciale headers gebeurd dan wordt het commando beschreven in de URL zelf.  
Dit gebeurd doormiddel van een _query_.  

Een _query_ is de gedeelte in een URL na de `?`.  
de query bestaat uit een _key_ en optioneel een _value_.  
key en value worden aan elkaar gekoppeld doormiddel van een `=`.  
Er kunnen ook meerdere queries in een url als de queries gesepereerd worden door een `&`.  
De keys en values hoeven niet uniek te zijn.

Voorbeeld van een valide URL:  
`https://www.web.site/path.html?key=value&key=value&only_key`

---

https://computer-database.gatling.io/computers is een statische demo website van [Gatling] en gebruikt parameters in de URL.  
In de notebook cells hieronder open we de demo website en bekijken we de Network tab voor XHR berichtenverkeer.  
En gaan we [requests] gebruiken om data van de webpagina op te vragen.  
We parsen de HTML met [beautifulsoup4].  

[Gatling]: https://gatling.io
[requests]: https://docs.python-requests.org/en/master/index.html "Requests: HTTP for Humans"
[beautifulsoup4]: https://beautiful-soup-4.readthedocs.io/en/latest/ "Beautiful Soup"

Open een browser en navigeer naar de demo pagina.  
Open de developer tools.  
Klik op de next knop op de demo pagina.
In de developer tools open kan er worden gezien dat er een GET request is geweest.  
De request en de URL in de browser hebben een query.  

> ![query string params](./img/devtools_query_params.png "Query String Parameters")

Met wat giswerk kan er worden bedacht wat de query bevat.

`p=0` de key `p` en de value als cijfer moet waarschijnlijk de pagina voorstellen.  
`n=10` de key `n` is hoogstwaarschijnlijk het aantal items in de [table] op de pagina.  
`s=name` de [table] is gesorteerd op naam, dus `s` is voor _sort_.  
`d=asc` moet dan voor de manier van sorteren zijn, `asc` staat voor _ascending_.

[table]: https://developer.mozilla.org/en-US/docs/Learn/HTML/Tables/Basics


Met deze kennis kunnen we een query maken voor een tabel met de benodigde regels.  
De URL + query kan dan met Python-requests worden opgevraagd.

In [8]:
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

In [9]:
base_url = 'https://computer-database.gatling.io/computers'
query = '?p={page}&n={rows}'
url = urljoin(base_url, query.format(page=0, rows=5))  # pagina 0, regels 5
url

'https://computer-database.gatling.io/computers?p=0&n=5'

De URL is gecreeerd en

In [10]:
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

Het is niet nodig maar notebook kan de HTML in de output cell weergeven.  
Het kan handig zijn om te zien wat de gekregen HTML representeerd.  
De HTML is zonder CSS opmaak dus kan er anders uit zien dan in de browser.  

In [11]:
# optioneel
from IPython import display
display.HTML(str(soup))

ACE,-,-,-
AN/FSQ-32,01 Jan 1960,-,IBM
AN/FSQ-7,01 Jan 1958,-,IBM
APEXC,-,-,-
ARRA,-,-,-


De data in de table van de HTML is nodig

In [12]:
table = soup.table
display.HTML(str(table))

ACE,-,-,-
AN/FSQ-32,01 Jan 1960,-,IBM
AN/FSQ-7,01 Jan 1958,-,IBM
APEXC,-,-,-
ARRA,-,-,-


Met een [list-comprehension] kan er  over de `<th>` tags geitereerd worden en zo uit elke tag de text filteren.  
Een list-comprehension is een loop in een data container zoals een `list`.  
De loop itereert en plaatst te objecten in een nieuwe data-container.  
list-comprehensions zijn efficient en flexibel.

[list-comprehension]: https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions

In [13]:
headers = [h.get_text() for h in table.thead.find_all('th')]
headers  # list met str objecten

['Computer name', 'Introduced', 'Discontinued', 'Company']

Nu er een lijst met headers is is er een indicatie welke cel wat bevat.  
De headers kan gebruikt worden als _key_ in een dict.  
De cellen in de regels zijn dan de _values_.  



In [14]:
rows = table.tbody.find_all('tr')
rows  # list met bs4.element.Tag objecten

[<tr><td><a href="/computers/381">ACE</a></td><td>-</td><td>-</td><td>-</td></tr>,
 <tr><td><a href="/computers/501">AN/FSQ-32</a></td><td>01 Jan 1960</td><td>-</td><td>IBM</td></tr>,
 <tr><td><a href="/computers/500">AN/FSQ-7</a></td><td>01 Jan 1958</td><td>-</td><td>IBM</td></tr>,
 <tr><td><a href="/computers/388">APEXC</a></td><td>-</td><td>-</td><td>-</td></tr>,
 <tr><td><a href="/computers/355">ARRA</a></td><td>-</td><td>-</td><td>-</td></tr>]

De headers en zijn de regels geparsed uit de table van de website.  
Nu kan deze data in een CSV file of JSON gezet worden.

Hieronder een voorbeeld hoe er een JSON van de data gemaakt kan worden.

In [15]:
import json

row_list = []

for row in rows:
    row_text = [r.get_text() for r in row]  # list comprehension om een list met text te maken
    # dict-zip
    header_row_dict = dict(zip(headers, row_text))
    row_list.append(header_row_dict)


# creeer de json
created_json = json.dumps({'rows': row_list}, indent=2)
print(created_json)

{
  "rows": [
    {
      "Computer name": "ACE",
      "Introduced": "-",
      "Discontinued": "-",
      "Company": "-"
    },
    {
      "Computer name": "AN/FSQ-32",
      "Introduced": "01 Jan 1960",
      "Discontinued": "-",
      "Company": "IBM"
    },
    {
      "Computer name": "AN/FSQ-7",
      "Introduced": "01 Jan 1958",
      "Discontinued": "-",
      "Company": "IBM"
    },
    {
      "Computer name": "APEXC",
      "Introduced": "-",
      "Discontinued": "-",
      "Company": "-"
    },
    {
      "Computer name": "ARRA",
      "Introduced": "-",
      "Discontinued": "-",
      "Company": "-"
    }
  ]
}


Hieronder een voorbeeld hoe de er een CSV file met de data gemaakt kan worden.

In [16]:
import csv

# csv filename
filename = 'gatling_io.csv'

# `with` context manager
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(headers)  # schrijf de header regel (fieldnames)
    for row in rows:
        row_text = [r.get_text() for r in row]
        writer.writerow(row_text)  # schrijf de regels


# open en lees de gemaakte file.
for line in open(filename, 'r').readlines():
    print(line.strip())

Computer name,Introduced,Discontinued,Company
ACE,-,-,-
AN/FSQ-32,01 Jan 1960,-,IBM
AN/FSQ-7,01 Jan 1958,-,IBM
APEXC,-,-,-
ARRA,-,-,-
